In [1]:
import django_initializer
from api.models.user.user import User
from django.contrib.auth.models import User as AuthUser
from json import loads, dumps
from collections import OrderedDict
import json
import re
from api.models.campaign.campaign import Campaign
from api.models.user.user_subscription import UserSubscription
def to_dict(input_ordered_dict):
    return loads(dumps(input_ordered_dict))

In [229]:
from django.conf import settings
import functools, logging, traceback
import os
import time
import ast
import json
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.color import Color
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import arrow


class FacebookSharedListCrawler():
    def __init__(self, page_username, target_post_id, chrome_driver_path=os.path.join(settings.BASE_DIR, 'chromedriver.exe')):
        self.chrome_options = Options() 
        # self.chrome_options.add_argument('--headless')  # 啟動Headless 無頭
        self.chrome_options.add_argument('--disable-gpu')
        
        self.chromeService = Service(executable_path=chrome_driver_path)

        self.driver = None #套用設定
        self.wait = None
        self.actions = None
        
        self.login_url = "https://m.facebook.com/"
        self.page_url = f'https://m.facebook.com/{page_username}/'
        self.target_post_id = target_post_id
        
        self.post_reference_id = ""
        self.post_shared_list_url = "https://m.facebook.com/browse/shares?id="
        
        self.email = "twadmin@algotech.app"
        self.password = "algoFB2021"
        self.loop_count = 0
        self.save_login = False
        self.is_login = False
        
    def create_driver(self):
        self.driver = webdriver.Chrome(options=self.chrome_options, service= self.chromeService)
        return self.driver
    
    def create_action_object(self):
        if self.driver:
            self.actions = ActionChains(self.driver)
        return self.actions
    
    def create_wait_object(self):
        if self.driver:
            self.wait = WebDriverWait(self.driver, 3)
        return self.wait
    
    def get_driver(self):
        return self.driver
    
    def login(self, validate=False):
        count = 0
        while not self.is_login and count <= 4:
            if not validate:
                self.driver.get(self.login_url)
            email_input = self.driver.find_element(By.ID, "m_login_email")
            pass_input = self.driver.find_element(By.ID, "m_login_password")
            login_button = self.driver.find_element(By.NAME, "login")
            self.actions.send_keys_to_element(email_input, self.email)
            self.actions.send_keys_to_element(pass_input, self.password)
            self.actions.click(login_button)
            self.actions.perform()
            count += 1
            self.is_login = True
        self.validate_user()
        
    def validate_user(self):
        print("validate_user")
        try:
            # check if page pops up log in with one tap
            self.driver.find_element(By.CSS_SELECTOR, "input[value='regular_login']")
            ok_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='OK']")
            self.actions.click(login_button).perform()
            self.save_login = True
        except NoSuchElementException:
            print("Element is not present")
            
        try:
            my_profile = self.driver.find_element(By.CSS_SELECTOR, f"div[role='button']")
            self.actions.click(my_profile).perform()

            pass_input = self.wait.until(
                EC.presence_of_element_located((By.NAME, "pass"))
            )
            self.actions.send_keys_to_element(pass_input, self.password)
            login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
            self.actions.click(login_button).perform()
        except NoSuchElementException:
            pass
    def expand_all(self):
        try:
            while True:
                see_more_button = self.driver.find_element(By.ID, "m_more_item")
                if see_more_button:
                    self.actions.click(see_more_button).perform()
        except NoSuchElementException:
            print("Element is not present")
            
    def collect_names(self):
        names = set([el.text for el in self.driver.find_elements(By.TAG_NAME, "strong")])
        return names
    
    def start(self):
        start = arrow.now()
        try:
            self.create_driver()
            self.create_wait_object()
            self.create_action_object()
            self.login()
            time.sleep(2)
            self.driver.get(self.page_url)
        
            while self.loop_count <= 3:
                self.loop_count += 1
                self.driver.execute_script("window.scrollTo(100,document.body.scrollHeight);")

                article = self.wait.until(
                        EC.presence_of_element_located((By.TAG_NAME, "article"))
                    )
                if article:
                    articles = self.driver.find_elements(By.XPATH, "//article")
                    for article in articles:
                        if self.post_reference_id:
                            break
                        post_data = json.loads(article.get_attribute("data-store"))
                        for key,value in post_data.items():
                            print(key)
                            print(value)
                            if key == "share_id":
                                if type(value) == int:
                                    post_id = str(value)
                                else:
                                    post_id = value.split(":")[1]
                                if post_id != self.target_post_id:
                                    break

                            if key == "feedback_target":
                                self.post_reference_id = str(value)
                                break

                self.loop_count += 1
            print(self.post_shared_list_url + self.post_reference_id)
            self.driver.get(self.post_shared_list_url + self.post_reference_id)
            if not self.save_login:
                self.validate_user()
            self.expand_all()
            name_list = self.collect_names()
            end = arrow.now()
            print(f"Spent: {end - start}")
#             self.driver.quit()
            return name_list
        except Exception as e:
            print(traceback.format_exc())
#             if self.driver:
#                 self.driver.quit()
            return None

In [230]:

page_username = "kuai.mai.2021"
target_post_id = "517999680166258"
fb_crawler = FacebookSharedListCrawler(page_username, target_post_id)
data = fb_crawler.start()
print(data)


validate_user
Element is not present
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_19580\4064349478.py", line 129, in start
    article = self.wait.until(
  File "C:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\liveshowseller-67dL19H--py3.9\lib\site-packages\selenium\webdriver\support\wait.py", line 90, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00D41ED3+2236115]
	Ordinal0 [0x00CD92F1+1807089]
	Ordinal0 [0x00BE66FD+812797]
	Ordinal0 [0x00C155DF+1005023]
	Ordinal0 [0x00C157CB+1005515]
	Ordinal0 [0x00C47632+1209906]
	Ordinal0 [0x00C31AD4+1120980]
	Ordinal0 [0x00C459E2+1202658]
	Ordinal0 [0x00C318A6+1120422]
	Ordinal0 [0x00C0A73D+960317]
	Ordinal0 [0x00C0B71F+964383]
	GetHandleVerifier [0x00FEE7E2+2743074]
	GetHandleVerifier [0x00FE08D4+2685972]
	GetHandleVerifier [0x00DD2BAA+532202]
	GetHandleVerifier [0x00DD1990+527568]
	Ordinal0